In [1]:
import requests
import pandas as pd
import bs4
from bs4 import BeautifulSoup as bs  
import scipy as sc
import numpy as np
import time

In [2]:
def Genres_Attributes_Fix(Anime_Page,Search_Word):
    g = Anime_Page.find('span',attrs={'class':'dark_text'},text=Search_Word).find_next()
    genres =""
    while g.name != 'div':
        if g.name == 'a':
            genres = genres+g.text+", "
        g = g.find_next()
    return genres[:-2]   


In [3]:
def Theme_Attributes_Fix(Anime_Page,Search_Word):
    g = Anime_Page.find('span',attrs={'class':'dark_text',},text=Search_Word)
    themes =""
    while g.name != 'div':
        if g.name == 'a':
            themes = themes+g.text+", "
        g = g.find_next()
    return themes[:-2] 

In [4]:
#####################################################################################
#####################################################################################
#####################################################################################
#####################################################################################
####put all the values in a list###
#We need to do it because some of the Animes have missing values like french names#
def Dict_To_List(dict,df_columns):
    Anime_Attributes = list()
    for i in df_columns:
        Anime_Attributes.append(dict.get(i))
    return Anime_Attributes


In [5]:
#####################################################################################
##getting the attributes from the anime page########################################
#####################################################################################
#####################################################################################
def get_Anime_Attributes(link):
    html = requests.get(link)
    Anime_Page = bs(html.text)
    Anime_Page.find_all("h2")
    Attribute_dict = {"English":Anime_Page.find("h1").strong.text,"Scored_By":Anime_Page.find('span', attrs={"itemprop":"ratingCount"}).text } 
    for heading in Anime_Page.find_all("h2"):
        if heading.text == "Alternative Titles" or heading.text == "Information" or heading.text == "Statistics":##to stop at the first heading
            tag = heading
            while tag.name != 'br' :                                                ###to stop at the end
                if tag.name == 'div' and tag.get('class')[0] == 'spaceit_pad':
                    
                    if tag.find('span',attrs={'class':'dark_text'}).text == "Favorites:":
                        Attribute_Name = tag.find('span',attrs={'class':'dark_text'}).text[:-1]##test
                        Attribute_Val = (tag.text.split(":",1)[1]).rstrip('\n').lstrip('\n').strip()
                        Attribute_dict[Attribute_Name] = Attribute_Val
                        return Attribute_dict
                    
                    elif tag.find('span',attrs={'class':'dark_text'}).text == "Genre:" or tag.find('span',attrs={'class':'dark_text'}).text == "Genres:":
                        word = tag.find('span',attrs={'class':'dark_text'}).text
                        Attribute_Name = "Genres"
                        Attribute_Val = Genres_Attributes_Fix(Anime_Page,word)
                    
                    elif tag.find('span',attrs={'class':'dark_text'}).text == "Theme:" or tag.find('span',attrs={'class':'dark_text'}).text == "Themes:":
                        word = tag.find('span',attrs={'class':'dark_text'}).text
                        Attribute_Name = "Theme"
                        Attribute_Val = Theme_Attributes_Fix(Anime_Page,word)
        
                    elif tag.find('span',attrs={'class':'dark_text'}).text == "Score:":
                        Score_Tag = tag.find('span',attrs={'class':'dark_text'})
                        Attribute_Name = tag.find('span',attrs={'class':'dark_text'}).text
                        if Score_Tag.find_next().text == 'N/A':
                            Attribute_Val = None 
                        else:    
                            Attribute_Val = Anime_Page.find('span',attrs={"itemprop":"ratingValue"}).text
        
                    elif tag.find('span',attrs={'class':'dark_text'}).text == "Ranked:":
                        Attribute_Name = tag.find('span',attrs={'class':'dark_text'}).text
                        Attribute_Val = Anime_Page.find('span', attrs={"class":"numbers ranked"}).strong.text[1:]

                    else:
                        Attribute_Name = tag.find('span',attrs={'class':'dark_text'}).text[:-1]##test
                        Attribute_Val = (tag.text.split(":",1)[1]).rstrip('\n').lstrip('\n').strip()
                               
                    Attribute_dict[Attribute_Name] = Attribute_Val         
                tag = tag.find_next()
                
    return Attribute_dict

In [6]:
#####################################################################################
##filling the data frame with the vlaue of the anime#################################
#####################################################################################
#####################################################################################
def filling_the_DataFrame(Start_Point,Link_To_Anime_Attribute_Page,Anime_df):
    for i in range(Start_Point,len(Link_To_Anime_Attribute_Page)):
            #if i != 0:
                 #print("running again")
            #Anime_df.to_csv("C:\\Users\\yakova\\Desktop\\Project\\DataFrame\\This DFs are from 2713\\MyAnimeList_Anime_Names_And_Links_df_0_"+str(i)+".csv", index=False)
            #time.sleep(60)
            #print("running again")
            try:
                link = Link_To_Anime_Attribute_Page.iat[i,1]
                anime_dict = get_Anime_Attributes(link)
                #if anime_dict == "We had and AttributeError":
                    #Anime_df.to_csv("test_DF_loop_v1.csv")
                    #time.sleep(300)
    
                anime_list = Dict_To_List(anime_dict,df_columns)
                Anime_df.loc[i] = anime_list
                if Anime_df.iloc[i,5] == None:
                    print("stopped at anime %d"%(i))
                    break
                else:
                    print(str(Anime_df.iloc[i,0])+" anime #%d is finised"%(i))

            except AttributeError:
                print("We had and AttributeError at anime #%d"%(i))
                print("Exporting the Data Frame")
                Anime_df.to_csv("C:\\Users\\yakova\\Desktop\\Project\\DataFrame_Final_v5\\MyAnimeList_Test_Crawler_0_"+str(i)+".csv", index=False)
                print("Starting 5min sleep time..")
                time.sleep(300)
                print("Finished 5min sleep time, starting loop again")
                filling_the_DataFrame(i,Link_To_Anime_Attribute_Page,Anime_df)
    print("The Data Frame is ready :)")
#return Anime_df
    
#####################################################################################
#####################################################################################
#####################################################################################
#####################################################################################    

In [7]:
#####################################################################################
##reading the CSV we created with all the anime links to a dataframe################
#####################################################################################
#####################################################################################
Link_To_Anime_Attribute_Page = pd.read_csv('.\MyAnimeList_Anime_Names_And_Links_df.csv')


In [8]:
#####################################################################################
##creating an empty dataframe with columns###########################################
#####################################################################################
#####################################################################################
link_for_columns_data = Link_To_Anime_Attribute_Page.iat[0,1] ##getting the link for the first anime to create the columns
df_columns = list(get_Anime_Attributes(link_for_columns_data).keys())##cast the dict to list and configure the df columns
Anime_df = pd.DataFrame(columns=[df_columns])##appling the columns to the df


In [ ]:
filling_the_DataFrame(0,Link_To_Anime_Attribute_Page,Anime_df)
Anime_df

Fullmetal Alchemist: Brotherhood anime #0 is finised
Gintama Season 4 anime #1 is finised
Attack on Titan Season 3 Part 2 anime #2 is finised
Steins;Gate anime #3 is finised
Fruits Basket: The Final anime #4 is finised
Gintama Season 2 anime #5 is finised
Hunter x Hunter anime #6 is finised
Gintama: Enchousen anime #7 is finised
Gintama Season 5 anime #8 is finised
March Comes In Like A Lion 2nd Season anime #9 is finised
Clannad ~After Story~ anime #10 is finised
Gintama anime #11 is finised
Attack on Titan Final Season Part 2 anime #12 is finised
Owarimonogatari Second Season anime #13 is finised
Code Geass: Lelouch of the Rebellion R2 anime #14 is finised
Gintama.: Silver Soul Arc - Second Half War anime #15 is finised
Attack on Titan: The Final Season anime #16 is finised
Ranking of Kings anime #17 is finised
Gintama.: Silver Soul Arc anime #18 is finised
Mob Psycho 100 II anime #19 is finised
Monster anime #20 is finised
Haikyu!! 3rd Season anime #21 is finised
Mushoku Tensei: Job

Naruto: Shippuden anime #179 is finised
SKET Dance anime #180 is finised
xxxHOLiC: Kei anime #181 is finised
Moriarty the Patriot anime #182 is finised
Dororo anime #183 is finised
Honey and Clover II anime #184 is finised
Fruits Basket 2019 1st Season anime #185 is finised
Golden Kamuy: Season 2 anime #186 is finised
Major S2 anime #187 is finised
Trigun anime #188 is finised
A Certain Scientific Railgun T anime #189 is finised
Bungou Stray Dogs 3rd Season anime #190 is finised
Horimiya anime #191 is finised
InuYasha: The Final Act anime #192 is finised
Little Busters! ~Refrain~ anime #193 is finised
Major S4 anime #194 is finised
86 anime #195 is finised
Bakuman. anime #196 is finised
Welcome to the Ballroom anime #197 is finised
Bungo Stray Dogs 2 anime #198 is finised
Chihayafuru anime #199 is finised
Fate/stay night [Unlimited Blade Works] anime #200 is finised
When They Cry: Kai anime #201 is finised
Nomad: Megalo Box 2 anime #202 is finised
Food Wars! Shokugeki no Soma anime #20

Full Metal Panic! The Second Raid anime #368 is finised
Searching for the Full Moon anime #369 is finised
JoJo's Bizarre Adventure anime #370 is finised
Akagi anime #371 is finised
Yakitate!! Japan anime #372 is finised
Yuru Yuri San☆Hai! anime #373 is finised
Sailor Moon Sailor Stars anime #374 is finised
Eyeshield 21 anime #375 is finised
Hanasaku Iroha: Blossoms for Tomorrow anime #376 is finised
Hidamari Sketch x Honeycomb anime #377 is finised
City Hunter anime #378 is finised
ef - a tale of memories. anime #379 is finised
Jormungand Season 2: Perfect Order anime #380 is finised
Kaleido Star anime #381 is finised
Macross anime #382 is finised
My Love Story!! anime #383 is finised
Overlord anime #384 is finised
Overlord III anime #385 is finised
Plastic Memories anime #386 is finised
Cromartie High School anime #387 is finised
Takarajima anime #388 is finised
TONIKAWA: Over the Moon For You anime #389 is finised
Ushio & Tora anime #390 is finised
Big Windup! anime #391 is finised
T

My Daddy Long Legs anime #558 is finised
Mobile Suit Gundam Wing anime #559 is finised
Princess Principal anime #560 is finised
Sasaki and Miyano anime #561 is finised
Seitokai Yakuindomo* anime #562 is finised
Sekai Ichi Hatsukoi - World's Greatest First Love anime #563 is finised
Armored Trooper Votoms anime #564 is finised
∀ Gundam anime #565 is finised
Aoi Bungaku Series anime #566 is finised
Baka & Test – Summon the Beasts 2 anime #567 is finised
Dragon Ball Z Kai anime #568 is finised
Ghost Stories anime #569 is finised
GATE anime #570 is finised
Wasteful Days of High School Girl anime #571 is finised
Anne of Green Gables anime #572 is finised
Good Luck Girl! anime #573 is finised
Sailor Moon anime #574 is finised
Sailor Moon R anime #575 is finised
Hayate the Combat Butler!! anime #576 is finised
The Idaten Deities Know Only Peace anime #577 is finised
Banner of the Stars anime #578 is finised
My Bride is a Mermaid anime #579 is finised
Space Pirate Captain Harlock anime #580 is

Moyashimon anime #736 is finised
Time of Shura anime #737 is finised
The Daughter of 20 Faces anime #738 is finised
No. 6 anime #739 is finised
Nobunaga Concerto anime #740 is finised
Haven't You Heard? I'm Sakamoto anime #741 is finised
Space Patrol Luluco anime #742 is finised
The Morose Mononokean II anime #743 is finised
Saiyuki anime #744 is finised
GUGURE! KOKKURI-SAN anime #745 is finised
Hinomaru Sumo anime #746 is finised
Love Live! Sunshine!! Season 2 anime #747 is finised
Ah! My Goddess: Flights of Fancy anime #748 is finised
Basilisk anime #749 is finised
Love, Chunibyo & Other Delusions!: Heart Throb anime #750 is finised
Fist of the North Star 2 anime #751 is finised
Hungry Heart: Wild Striker anime #752 is finised
BOFURI: I Don't Want to Get Hurt, so I'll Max Out My Defense. anime #753 is finised
Junjo Romantica anime #754 is finised
Minami-ke Okaeri anime #755 is finised
Mitsudomoe anime #756 is finised
Mobile Fighter G Gundam anime #757 is finised
Over Drive anime #758

...

Tondemo Senshi Muteking anime #3246 is finised
Triton of the Sea anime #3247 is finised
Nanami-chan 2nd Series anime #3248 is finised
Saban's Adventures of the Little Mermaid anime #3249 is finised
Hamos: The Green Chariot anime #3250 is finised
Soccer Fever anime #3251 is finised
THE IDOLM@STER SideM Wakeatte Mini! anime #3252 is finised
Tono to Issho: Ippunkan Gekijou anime #3253 is finised
Space Emperor God Sigma anime #3254 is finised
Waccha PriMagi! anime #3255 is finised
Wooser's Hand-to-Mouth Life anime #3256 is finised
Pleasant Goat and Big Big Wolf anime #3257 is finised
Zoids Wild anime #3258 is finised
Animentary: Ketsudan anime #3259 is finised
Chu-Bra anime #3260 is finised
Galactic Patrol Lensman anime #3261 is finised
Galactic Gale Baxingar anime #3262 is finised
Gregory Horror Show: The Last Train anime #3263 is finised
Aria the Scarlet Ammo AA anime #3264 is finised
Juuou Mujin no Fafnir anime #3265 is finised
Bakugan: Mechtanium Surge anime #3266 is finised
Barbapapa 

Robotan anime #3420 is finised
Farewell, My Dear Cramer anime #3421 is finised
Wings of Dragon anime #3422 is finised
Tama & Friends: Uchi no Tama Shirimasen ka? anime #3423 is finised
Zoku Attacker You! Kin Medal e no Michi anime #3424 is finised
Osamake: Romcom Where The Childhood Friend Won't Lose anime #3425 is finised
Ape Escape anime #3426 is finised
Seraphim Call anime #3427 is finised
Yumeria anime #3428 is finised
Aquarian Age: Sign for Evolution anime #3429 is finised
Adventures of the Monkey King anime #3430 is finised
Instant History anime #3431 is finised
Kinpatsu no Jeanie anime #3432 is finised
Knyacki! anime #3433 is finised
Manga Hajimete Monogatari anime #3434 is finised
Cybuster anime #3435 is finised
Balala the Faries: Over the Rainbow anime #3436 is finised
Bakukyuu Hit! Crash B-Daman anime #3437 is finised
Chinpui anime #3438 is finised
Ai-Mai-Mi Surgical Friends anime #3439 is finised
Chou Supercar Gattiger anime #3440 is finised
DIVE!! anime #3441 is finised
Leg

Ryuusei Sentai Musumet anime #3599 is finised
Suzy's Zoo: Daisuki! Witzy anime #3600 is finised
Time Bokan 2000: Kaitou Kiramekiman anime #3601 is finised
Wind: A Breath of Heart (TV) anime #3602 is finised
Yamishibai: Japanese Ghost Stories 7 anime #3603 is finised
Ba La La Xiao Mo Xian: Caihong Xin Shi anime #3604 is finised
Dorei-ku The Animation anime #3605 is finised
Duel Masters Zero anime #3606 is finised
Fushigi Dagashiya: Zenitendou anime #3607 is finised
Bouken Korobokkuru anime #3608 is finised
Chogattai Majutsu Robot Ginguiser anime #3609 is finised
DD Fist of the North Star 2: Strawberry Flavor Plus anime #3610 is finised
Gu-Gu Ganmo anime #3611 is finised
Haitai Nanafa anime #3612 is finised
Haitai Nanafa 2nd Season anime #3613 is finised
Blazing Teens 3 anime #3614 is finised
Samurai Girl Real Bout High School anime #3615 is finised
Saru Getchu: On Air 2nd anime #3616 is finised
Time Bokan: The Villains Strike Back anime #3617 is finised
Tomica Kizuna Gattai: Earth Grann

Eternity: Sweet Love Story anime #3780 is finised
Genki!! Ekoda-chan anime #3781 is finised
Hello Kitty: Ringo no Mori no Mystery anime #3782 is finised
The Affectuous Family anime #3783 is finised
Straight Title Robot Anime anime #3784 is finised
Fruity Samurai anime #3785 is finised
Mighty Orbots anime #3786 is finised
TAMAYOMI: The Baseball Girls anime #3787 is finised
Zenmai Zamurai anime #3788 is finised
Pororo the Little Penguin 2 anime #3789 is finised
Rebirth anime #3790 is finised
Sweet Valerian anime #3791 is finised
Aesop's World anime #3792 is finised
Aquarion Logos anime #3793 is finised
Babel II: Beyond Infinity anime #3794 is finised
Bakumatsu anime #3795 is finised
Battery anime #3796 is finised
Duel Masters (2017) anime #3797 is finised
Lemon Angel (1988) anime #3798 is finised
Pugyuru anime #3799 is finised
Stand My Heroes: Piece of Truth anime #3800 is finised
Schoolgirl Strikers: Animation Channel anime #3801 is finised
Shounen Ashibe: Go! Go! Goma-chan 3 anime #380

SD Gundam Force anime #3967 is finised
Alice or Alice anime #3968 is finised
Lord of Vermilion: The Crimson King anime #3969 is finised
Virtual-san Looking anime #3970 is finised
Onigiri anime #3971 is finised
Ulysses: Jeanne d'Arc and the Alchemist Knight anime #3972 is finised
Venus Project: Climax anime #3973 is finised
Beast Fighter: The Apocalypse anime #3974 is finised
Makeruna!! Aku no Gundan! anime #3975 is finised
Kara The Animation anime #3976 is finised
Maji de Otaku na English! Ribbon-chan: Eigo de Tatakau Mahou Shoujo - The TV anime #3977 is finised
Shironeko Project ZERO CHRONICLE anime #3978 is finised
Minarai Diva anime #3979 is finised
Bottom Biting Bug 2 anime #3980 is finised
Mazica Party anime #3981 is finised
Nobunaga teacher's young bride anime #3982 is finised
Doamayger-D anime #3983 is finised
Hand Shakers anime #3984 is finised
Mini Moni Yaru no da Pyon! anime #3985 is finised
Akiba-chan anime #3986 is finised
MapleStory anime #3987 is finised
Bottom Biting Bug

In [ ]:
url = Link_To_Anime_Attribute_Page.iloc[0,1]
get_Anime_Attributes(url)


In [ ]:
print(page.find('span', attrs={"class":"numbers ranked"}).strong.text[1:])##getting the rank
print(page.find('span', attrs={"itemprop":"ratingCount"}).text)##getting the "Scored_By"



In [ ]:
def filling_the_Rank_and_ScoredBy_Arr(Start_Point,Link_To_Anime_Attribute_Page,Anime_df):
    arr_rank = list()
    arr_scoredby = list()
    for i in range(Start_Point,len(Link_To_Anime_Attribute_Page)):
        test_url = Link_To_Anime_Attribute_Page.iloc[i,1]
        tes_html = requests.get(test_url)
        page = bs(tes_html.text)
        arr_rank.append(page.find('span', attrs={"class":"numbers ranked"}).strong.text[1:])##getting the rank
        arr_scoredby.append(page.find('span', attrs={"itemprop":"ratingCount"}).text)
        print(str(Link_To_Anime_Attribute_Page.iloc[i,0])+" anime #%d is finised"%(i))
    
    return arr_rank,arr_scoredby

In [ ]:
Rank_and_ScoredBy_df = pd.DataFrame(columns=["English","Ranked","Scored_By"])
rank_list = list("")
scoredby_list = list("")
rank_list,scoredby_list = filling_the_Rank_and_ScoredBy_Arr(0,Link_To_Anime_Attribute_Page,Rank_and_ScoredBy_df)
rank_list

In [ ]:
test_anime = get_Anime_Attributes(test_url)
test_anime
class=
